In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
train = pd.read_csv('/content/gdrive/MyDrive/replicated_results_train_full.csv')
valid = pd.read_csv('/content/gdrive/MyDrive/replicated_results_valid_full.csv')
train.info()
valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3488 entries, 0 to 3487
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   text             3488 non-null   object 
 1   label            3488 non-null   object 
 2   PredictedLabel   3488 non-null   object 
 3   ConfidenceScore  3488 non-null   float64
dtypes: float64(1), object(3)
memory usage: 109.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   text             388 non-null    object 
 1   label            388 non-null    object 
 2   PredictedLabel   388 non-null    object 
 3   ConfidenceScore  388 non-null    float64
dtypes: float64(1), object(3)
memory usage: 12.2+ KB


In [ ]:
# Reset the index of each DataFrame
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)

# Concatenate the DataFrames
train_all = pd.concat([train, valid]).reset_index(drop=True)

df_incorrect = train_all[train_all['label'] != train_all['PredictedLabel']]
df_correct = train_all[train_all['label'] == train_all['PredictedLabel']]
df_incorrect = df_incorrect.reset_index(drop=True)
df_correct = df_correct.reset_index(drop=True)

In [ ]:
print(df_correct.info())
print(df_incorrect.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3494 entries, 0 to 3493
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   text             3494 non-null   object 
 1   label            3494 non-null   object 
 2   PredictedLabel   3494 non-null   object 
 3   ConfidenceScore  3494 non-null   float64
dtypes: float64(1), object(3)
memory usage: 109.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382 entries, 0 to 381
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   text             382 non-null    object 
 1   label            382 non-null    object 
 2   PredictedLabel   382 non-null    object 
 3   ConfidenceScore  382 non-null    float64
dtypes: float64(1), object(3)
memory usage: 12.1+ KB
None


In [ ]:
df_correct.label.value_counts()

label
neutral     2003
positive    1026
negative     465
Name: count, dtype: int64

In [ ]:
df_incorrect.label.value_counts()

label
neutral     301
positive     70
negative     11
Name: count, dtype: int64

In [ ]:
%pip install openai --upgrade
from openai import OpenAI
client = OpenAI(api_key='sk-proj-hEv32vgqXTnk8oxTyhhpT3BlbkFJSBixmeuoz6s5mkpOg9NU',)
import pandas as pd
import re  # Import the regex module

In [ ]:
def generate_financial_statements(text, label):
    system_prompt = f'''
    Acting as a seasoned financial analyst, generate 10 variations of the provided financial news statement while retaining the "{label}" sentiment and financial context.
    Incorporate diverse language and sentence structures, and integrate domain-specific terminology to ensure authenticity and relevance within the financial sector.
    Aim for variations of similar length and complexity to the original sentence.
    Please avoid using unreadable characters. Provide only the 10 numbered statements without explanations.
    You will output a json object containing the following information:

    {{
      "variations": string[] // Array of 10 variations based on the provided financial news statement
    }}
    '''

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": text
            }
        ],
        response_format={
        "type": "json_object"
        },
        temperature=0.7,
        max_tokens=1000,
        top_p=1,
        frequency_penalty=0.5,
        presence_penalty=0
    )

    return response.choices[0].message.content

In [ ]:
# # Testing on a few examples
# for _, row in df_incorrect[:5].iterrows():
#     result = generate_financial_statements(row['text'],row['label'])
#     print(f"Text: {row['text']}\nLabel: {row['label']}\n\nRESULT: {result}")
#     print("\n\n----------------------------\n\n")

Text: Following the transaction , Lundbeck has worldwide rights for nalmefene , excluding North America , Mexico , Turkey and South-Korea .
Label: positive

RESULT: {
    "variations": [
        "Post the deal, Lundbeck secures exclusive global licensing rights for nalmefene, with the exception of North America, Mexico, Turkey, and South Korea.",
        "Lundbeck gains global exclusivity for nalmefene post-transaction, except for North America, Mexico, Turkey, and South Korea.",
        "Upon completion of the transaction, Lundbeck attains worldwide rights for nalmefene, excluding North America, Mexico, Turkey, and South Korea.",
        "Following the agreement, Lundbeck obtains global licensing rights for nalmefene while excluding North America, Mexico, Turkey, and South Korea.",
        "After the transaction is finalized, Lundbeck holds worldwide rights for nalmefene except for North America, Mexico, Turkey, and South Korea.",
        "Lundbeck acquires global exclusivity for nalm

In [ ]:
# Creating an array of json tasks

tasks = []

# for index, row in df_incorrect.iterrows():

#     description = row['text']
#     system_prompt = f'''
#     Acting as a seasoned financial analyst, generate 20 variations of the provided financial news statement while retaining the "{row['label']}" sentiment and financial context.
#     Incorporate diverse language and sentence structures, and integrate domain-specific terminology to ensure authenticity and relevance within the financial sector.
#     Aim for variations of similar length and complexity to the original sentence.
#     Please avoid using unreadable characters. Provide only the 20 statements without explanations.
#     You will output a json object containing the following information:

#     {{
#       "statements": string[] // Array of 20 variations based on the provided financial news statement
#     }}
#     '''
#     task = {
#         "custom_id": f"task_incorrect1-{index}",
#         "method": "POST",
#         "url": "/v1/chat/completions",
#         "body": {
#             # This is what you would have in your Chat Completions API call
#             "model": "gpt-3.5-turbo",
#             "temperature": 0.7,
#             "max_tokens":2000,
#             "top_p":1,
#             "frequency_penalty":0.5,
#             "presence_penalty":0,
#             "response_format": {
#                 "type": "json_object"
#             },
#             "messages": [
#                 {
#                     "role": "system",
#                     "content": system_prompt
#                 },
#                 {
#                     "role": "user",
#                     "content": description
#                 }
#             ],
#         }
#     }

#     tasks.append(task)

# for index, row in df_incorrect.iterrows():

#   description = row['text']
#   system_prompt = f'''
#   Acting as a seasoned financial analyst, generate 30 different financial news statements similar to the provided financial news statement and having a "{row['label']}" sentiment.
#   Incorporate diverse language and sentence structures, and integrate domain-specific terminology to ensure authenticity and relevance within the financial sector.
#   Aim for financial news statements of similar length and complexity to the original sentence.
#   Please avoid using unreadable characters. Provide only the 30 statements without explanations.
#   You will output a JSON object containing the following information:

#   {{
#     "statements": string[] // Array of 30 statements based on the provided financial news statement
#   }}
#   '''

#   task = {
#       "custom_id": f"_incorrect2-{index}",
#       "method": "POST",
#       "url": "/v1/chat/completions",
#       "body": {
#           # This is what you would have in your Chat Completions API call
#           "model": "gpt-3.5-turbo",
#           "temperature": 0.7,
#           "max_tokens":3000,
#           "top_p":1,
#           "frequency_penalty":0.5,
#           "presence_penalty":0,
#           "response_format": {
#               "type": "json_object"
#           },
#           "messages": [
#               {
#                   "role": "system",
#                   "content": system_prompt
#               },
#               {
#                   "role": "user",
#                   "content": description
#               }
#           ],
#       }
#   }

#   tasks.append(task)

for index, row in df_correct[3000:].iterrows():

    description = row['text']
    system_prompt = f'''
    Acting as a seasoned financial analyst, generate 5 variations of the provided financial news statement while retaining the "{row['label']}" sentiment and financial context.
    Incorporate diverse language and sentence structures, and integrate domain-specific terminology to ensure authenticity and relevance within the financial sector.
    Aim for variations of similar length and complexity to the original sentence.
    Please avoid using unreadable characters. Provide only the 5 statements without explanations.
    You will output a json object containing the following information:

    {{
      "statements": string[] // Array of 5 variations based on the provided financial news statement
    }}
    '''
    task = {
        "custom_id": f"task_correct-{index}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            # This is what you would have in your Chat Completions API call
            "model": "gpt-3.5-turbo",
            "temperature": 0.7,
            "max_tokens":500,
            "top_p":1,
            "frequency_penalty":0.5,
            "presence_penalty":0,
            "response_format": {
                "type": "json_object"
            },
            "messages": [
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": description
                }
            ],
        }
    }

    tasks.append(task)

In [ ]:
# Creating the file
import json
file_name = "/content/gdrive/MyDrive/batch_tasks_augmentation8.jsonl"

with open(file_name, 'w') as file:
    for obj in tasks:
        file.write(json.dumps(obj) + '\n')

In [ ]:
df = pd.read_json(file_name, lines=True)

# Display the first 10 records
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494 entries, 0 to 493
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   custom_id  494 non-null    object
 1   method     494 non-null    object
 2   url        494 non-null    object
 3   body       494 non-null    object
dtypes: object(4)
memory usage: 15.6+ KB


In [ ]:
pd.set_option('display.max_colwidth', None)
print(df['body'].head(3))

0                                                                                     {'model': 'gpt-3.5-turbo', 'temperature': 0.7000000000000001, 'max_tokens': 500, 'top_p': 1, 'frequency_penalty': 0.5, 'presence_penalty': 0, 'response_format': {'type': 'json_object'}, 'messages': [{'role': 'system', 'content': '
    Acting as a seasoned financial analyst, generate 5 variations of the provided financial news statement while retaining the "neutral" sentiment and financial context.
    Incorporate diverse language and sentence structures, and integrate domain-specific terminology to ensure authenticity and relevance within the financial sector.
    Aim for variations of similar length and complexity to the original sentence.
    Please avoid using unreadable characters. Provide only the 5 statements without explanations.
    You will output a json object containing the following information:

    {
      "statements": string[] // Array of 5 variations based on the provided financial ne

In [ ]:
batch_file = client.files.create(
  file=open(file_name, "rb"),
  purpose="batch"
)

In [ ]:
print(batch_file)

FileObject(id='file-KIDWM8yHhqEXKLfannBreEXb', bytes=590483, created_at=1717883418, filename='batch_tasks_augmentation8.jsonl', object='file', purpose='batch', status='processed', status_details=None)


In [ ]:
batch_job = client.batches.create(
  input_file_id=batch_file.id,
  endpoint="/v1/chat/completions",
  completion_window="24h"
)

In [ ]:
batch_job = client.batches.retrieve(batch_job.id)
print(batch_job)

Batch(id='batch_i2DMzWyQGyD4zCS1yY8JBoBC', completion_window='24h', created_at=1717883425, endpoint='/v1/chat/completions', input_file_id='file-KIDWM8yHhqEXKLfannBreEXb', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1717883563, error_file_id=None, errors=None, expired_at=None, expires_at=1717969825, failed_at=None, finalizing_at=1717883541, in_progress_at=1717883426, metadata=None, output_file_id='file-YiQuzddscRetejhqKjdbie8U', request_counts=BatchRequestCounts(completed=494, failed=0, total=494))


In [ ]:
result_file_id = batch_job.output_file_id
result = client.files.content(result_file_id).content

In [ ]:
result_file_name = "/content/gdrive/MyDrive/batch_job_results_correct_part7.jsonl"

with open(result_file_name, 'wb') as file:
    file.write(result)

In [ ]:
df = pd.read_json(result_file_name, lines=True)

# Display the first 10 records
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494 entries, 0 to 493
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         494 non-null    object 
 1   custom_id  494 non-null    object 
 2   response   494 non-null    object 
 3   error      0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 15.6+ KB


In [ ]:
print(df['response'].head(3))

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              {'status_code': 200, 'request_id': 'f86a4277c4eb973dd62a79ed5730b970', 'body': {'id': 'chatcmpl-9XyG6phGa

In [ ]:
# Define a function to safely parse and extract statements
def extract_statements(row):
    try:
        # Accessing the nested 'choices' array correctly based on your structure
        choices = row['body']['choices']
        # Extract the content from the first choice's message
        content = json.loads(choices[0]['message']['content'])
        return content['statements']
    except (KeyError, TypeError, json.JSONDecodeError) as e:
        print(f"Error parsing row: {e}")  # Print the error for debugging
        return []  # Return an empty list in case of other errors

# Apply the function to extract statements
df['statements'] = df['response'].apply(extract_statements)

# Explode the 'statements' column into multiple rows
expanded_df = df.explode('statements')

# Reset the index if desired
expanded_df.reset_index(drop=True, inplace=True)

# Display the expanded DataFrame
print(expanded_df[['statements']])


                                                                                                                                                                                           statements
0                           The recently conducted market analysis offers a comprehensive insight into the current landscape and future prospects of the pharmaceutical sector in the Czech Republic.
1                                        A fresh market analysis has provided a detailed summary of the present status and forthcoming trends in the pharmaceutical market within the Czech Republic.
2                       This newly released market analysis presents a comprehensive view of both the current scenario and the future projections of the pharmaceutical market in the Czech Republic.
3                                                        An up-to-date market analysis sheds light on the current state and upcoming developments in the pharmaceutical sector of the Czech Republic.
4         

In [ ]:
new_df = expanded_df[['statements']].copy()

In [ ]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   statements  2478 non-null   object
dtypes: object(1)
memory usage: 19.5+ KB


In [ ]:
new_df.head()

,statements
0,The recently conducted market analysis offers a comprehensive insight into the current landscape and future prospects of the pharmaceutical sector in the Czech Republic.
1,A fresh market analysis has provided a detailed summary of the present status and forthcoming trends in the pharmaceutical market within the Czech Republic.
2,This newly released market analysis presents a comprehensive view of both the current scenario and the future projections of the pharmaceutical market in the Czech Republic.
3,An up-to-date market analysis sheds light on the current state and upcoming developments in the pharmaceutical sector of the Czech Republic.
4,The latest market analysis delivers a thorough examination of both the existing conditions and future expectations within the pharmaceutical market of Czech Republic.


In [ ]:
new_df.to_csv('/content/gdrive/MyDrive/incorrect_labelled_dataset_part7_0806.csv', index=False)

In [ ]:
# Base path to the files
base_path = '/content/gdrive/MyDrive/'

# List to hold each DataFrame
dataframes = []

# Loop through the range of file numbers
for i in range(1, 8):
    # Generate the file path
    file_path = f'{base_path}incorrect_labelled_dataset_part{i}_0806.csv'

    # Read the CSV file
    df = pd.read_csv(file_path)

    # Append the DataFrame to the list
    dataframes.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Display the first few rows of the combined DataFrame to verify
print(combined_df.head())


                                                                                                                                                                                                                statements
0           Following the reporting period, Somaxon Pharmaceuticals, BioTie's North American licensing partner, disclosed favorable outcomes from a preliminary Phase 2 clinical study on nalmefene for smoking cessation.
1                  Post the reporting period, Somaxon Pharmaceuticals, the North American licensee of BioTie, unveiled encouraging findings from a pilot Phase 2 clinical trial involving nalmefene for smoking cessation.
2  After the reporting period, positive results were revealed by Somaxon Pharmaceuticals, BioTie's licensing partner in North America, following a pilot Phase 2 clinical trial assessing nalmefene for smoking cessation.
3         In the aftermath of the reporting period, Somaxon Pharmaceuticals shared optimistic results from a pilot Phase 2 c

In [ ]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17462 entries, 0 to 17461
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   statements  17458 non-null  object
dtypes: object(1)
memory usage: 136.5+ KB


In [ ]:
combined_df.to_csv('/content/gdrive/MyDrive/FinbertMatch_unlabelled_dataset_ALL_0806.csv', index=False)

In [ ]:
data = pd.read_csv('/content/gdrive/MyDrive/incorrect_unlabelled_dataset_0806.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13198 entries, 0 to 13197
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   statements  13148 non-null  object
dtypes: object(1)
memory usage: 103.2+ KB
